In [2]:
import pandas as pd
import pandas_ta as ta

In [4]:
df = pd.read_csv('SET_DLY_ICHI_MA9,D.csv')

In [6]:
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1066,21.702183,23.306204,12.613046
1067,22.278455,22.420783,12.133867
1068,22.295488,21.124386,13.359647
1069,21.814464,19.886097,14.530482


In [12]:
df.to_csv('SET_DLY_ICHI_IND.csv', index=False)

In [9]:
df.ta.rsi(append=True)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
1066    61.642682
1067    61.642682
1068    54.805689
1069    58.139077
1070    53.860885
Name: RSI_14, Length: 1071, dtype: float64

In [11]:
df

,time,open,high,low,close,MA,Smoothing Line,Volume,Volume MA,ADX_14,DMP_14,DMN_14,RSI_14
0,2018-11-05T02:00:00Z,3.487277,3.487277,3.435993,3.435993,3.441692,3.462585,190600,870980.00,NaN,NaN,NaN,NaN
1,2018-11-06T02:00:00Z,3.435993,3.453088,3.401804,3.401804,3.439792,3.450809,463500,842750.00,NaN,NaN,NaN,NaN
2,2018-11-07T02:00:00Z,3.453088,3.470182,3.401804,3.453088,3.435993,3.443591,296100,776440.00,NaN,NaN,NaN,NaN
3,2018-11-08T02:00:00Z,3.453088,3.470182,3.435993,3.435993,3.432195,3.439792,142200,751735.00,NaN,NaN,NaN,NaN
4,2018-11-09T02:00:00Z,3.435993,3.453088,3.384710,3.401804,3.428396,3.435613,890600,771445.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,2023-03-28T02:00:00Z,13.200000,13.400000,13.200000,13.400000,13.111111,12.997778,3045594,9816069.10,21.702183,23.306204,12.613046,61.642682
1067,2023-03-29T02:00:00Z,13.400000,13.400000,13.200000,13.400000,13.155556,13.055556,2345537,9193328.75,22.278455,22.420783,12.133867,61.642682
1068,2023-03-30T02:00:00Z,13.300000,13.400000,13.100000,13.100000,13.166667,13.106667,2626446,8446174.15,22.295488,21.124386,13.359647,54.805689
1069,2023-03-31T02:00:00Z,13.200000,13.300000,13.000000,13.300000,13.233333,13.151111,3787206,8225572.90,21.814464,19.886097,14.530482,58.139077


In [2]:
import vectorbtpro as vbt

/opt/anaconda3/envs/deepq/lib/python3.8/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [3]:
class Port:
    def __init__(
            self,
            cash :float
    ) -> None:
        self.port = vbt.pf_enums.ExecState(
            cash=cash,
            position=0.0,
            debt=0.0,
            locked_cash=0.0,
            free_cash=cash,
            val_price=0.0,
            value=0.0
        )

    def get_current_cash(self) -> float:
        return self.port.free_cash
    
    def get_current_position(self) -> float:
        return self.port.position
    
    def create_order(
        self,
        size: float,
        price: float,
        direction: float,
        fees : float
    ) -> vbt.portfolio.enums.Order:

        order = vbt.pf_nb.order_nb(
            size=size,
            price=price,
            direction=direction,
            size_granularity=1.0,
            log=True,
            fees=fees
        )
        return order

    def process_order(self,order) -> None:
        order_result, self.port = vbt.pf_nb.process_order_nb(
            0, 0, 0,
            exec_state=self.port,
            order=order,
            update_value=True,
        )
        return order_result

    def open_long(self,price : float,size : float,fees :float) -> None:
        order = self.create_order(
            size=size,
            price=price,
            direction=0,
            fees=fees
        )
        self.process_order(order)

    def open_short(self,price : float,size : float,fees :float) -> None:
        order = self.create_order(
            size=size,
            price=price,
            direction=1,
            fees=fees
        )
        self.process_order(order)

    def close_long(self,price : float,fees :float) -> None:
        order = self.create_order(
            size= -self.get_current_position(),
            price=price,
            direction=0,
            fees=fees
        )
        p = self.process_order(order)

        return p
        

    def close_short(self,price : float,fees :float) -> None:
        order = self.create_order(
            size=self.get_current_position(),
            price=price,
            direction=1,
            fees=fees
        )
        self.process_order(order)
    
    def get_profit(self) -> float:
        return self.port.value


In [15]:
port = Port(cash=30.0)
print(port.port)


ExecState(cash=30.0, position=0.0, debt=0.0, locked_cash=0.0, free_cash=30.0, val_price=0.0, value=0.0)


In [19]:
print("- open long -")
port.open_long(price=1.0,size=1.0,fees=0.01)
print(port.port.position)



- open long -
4.0


In [12]:
print("- close long -")
port.close_long(price=100.0,fees=0.01)
print(port.port)

- close long -
ExecState(cash=30.0, position=0.0, debt=0.0, locked_cash=0.0, free_cash=30.0, val_price=0.0, value=0.0)


In [13]:
print("- current cash -")
print(port.get_current_cash())

- current cash -
30.0


In [14]:
port.get_current_cash

<bound method Port.get_current_cash of <__main__.Port object at 0x7fbbdb1b0af0>>

In [58]:
port.port.position

0.0